## 사전학습된 klue/bert-base 사용하기
* colab 으로 실습을 권장합니다.

In [ ]:
!nvidia-smi

[huggingface/datasets: 🤗 The largest hub of ready-to-use datasets for ML models with fast, easy-to-use and efficient data manipulation tools](https://github.com/huggingface/datasets)

In [ ]:
# transformers는 Hugging Face에서 제공하는 자연어처리 라이브러리로, 다양한 사전학습 언어모델을 제공하고 있습니다.
# accelerate는 Hugging Face에서 제공하는 학습 가속 라이브러리로, PyTorch와 함께 사용할 수 있습니다.
# accelerate를 이용하면 단일 노드의 다중 GPU 학습을 지원하고, 학습시간을 단축할 수 있습니다.
!pip install transformers --upgrade --q
!pip install accelerate --q
!pip install datasets --q

In [ ]:
# 코드 실행에 걸린 시간을 확인하기 위해
import time
start_time = time.time()
start_time

In [ ]:
import os
import warnings

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import datasets
from datasets import load_metric
import torch
from torch.utils.data import Dataset, DataLoader

warnings.filterwarnings(action='ignore')

In [ ]:
# 데이콘의 해당 데이터셋은 CC-BY-4.0 라이센스입니다.
# 데이터 출처 : https://dacon.io/competitions/official/235747/data
# 로컬 PC에서 실습 시 직접 데이콘 사이트에 회원가입하고 다운로드 해주세요.

import os, platform

base_path = "data/klue"
file_name = "dacon-klue-open-zip"

def file_exist_check(base_path):
    if os.path.exists(f"{base_path}train_data.csv"):
        print(f"{os.getcwd()}/{base_path} 경로에 파일이 있음")
        return

    if not os.path.exists(base_path):
        os.makedirs(base_path)

    if platform.system() == "Linux":
        print(f"파일을 다운로드 하고 {base_path} 경로에 압축을 해제함")
        !wget https://bit.ly/{file_name}
        !unzip {file_name} -d {base_path}
        return
    else:
        print(f"""https://dacon.io/competitions/official/235747/data 에서 다운로드 하고
              실습 경로 {os.getcwd()}/{base_path}에 옮겨주세요.""")
        return

file_exist_check(base_path)

In [ ]:
train = pd.read_csv("data/klue/train_data.csv")
test = pd.read_csv("data/klue/test_data.csv")

In [ ]:
# train_test_split
# dataset_train, dataset_val


## BERTDataset

* PyTorch의 Dataset 클래스를 상속받아 BERT 모델 학습을 위한 데이터셋을 정의
    * __init__ 메서드: 데이터셋을 초기화합니다. 주요 인자로는 dataset (데이터프레임 또는 유사한 형태의 데이터), sent_key (문장을 나타내는 열의 키), label_key (라벨을 나타내는 열의 키), 그리고 BERT 토크나이저(bert_tokenizer)가 있습니다.

    * sentences: 입력 문장들을 BERT 토크나이저를 사용하여 토큰화한 결과를 저장하는 리스트입니다.
    * labels: 라벨을 저장하는 리스트로, 학습 데이터일 경우에만 해당합니다.
    * mode: 데이터셋이 학습인지(test)인지를 나타내는 문자열 변수입니다.
    * __getitem__ 메서드: 주어진 인덱스 i에 대한 데이터를 반환합니다. 학습 모드인 경우에는 문장과 해당 라벨을 포함한 딕셔너리를 반환하고, 테스트 모드인 경우에는 단순히 문장을 반환합니다.

    * __len__ 메서드: 데이터셋의 전체 길이를 반환합니다.

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_key, label_key, bert_tokenizer):

        self.sentences = [
            bert_tokenizer(i,
                           truncation=True,
                           return_token_type_ids=False
                           ) for i in dataset[sent_key] ]

        if not label_key == None:
            self.mode = "train"
        else:
            self.mode = "test"

        if self.mode == "train":
            self.labels = [np.int64(i) for i in dataset[label_key]]
        else:
            self.labels = [np.int64(0) for i in dataset[sent_key]]

    def __getitem__(self, i):
        if self.mode == "train":
            self.sentences[i]["label"] = self.labels[i]
            return self.sentences[i]

        else:
            return self.sentences[i]

    def __len__(self):
        return (len(self.labels))


## klue/bert-base
klue/bert-base 모델 사용

* https://huggingface.co/klue/bert-base
* https://huggingface.co/docs/transformers/model_doc/auto


* AutoTokenizer: 
    * 모델에 대한 토크나이저를 자동으로 선택하고 불러오는 클래스입니다. 
    * from_pretrained 메서드를 사용하여 미리 학습된 모델의 토크나이저를 가져올 수 있습니다.
* AutoModelForSequenceClassification: 
    * 시퀀스 분류 작업을 위한 모델을 자동으로 선택하고 불러오는 클래스입니다. 
    * from_pretrained 메서드를 사용하여 미리 학습된 모델을 가져올 수 있습니다. 
    * 이 모델은 텍스트 시퀀스를 입력으로 받아, 시퀀스의 분류 레이블을 예측하는 작업에 사용됩니다.
* TrainingArguments: 
    * 모델을 학습시키기 위한 여러 인자들을 설정하는 클래스입니다. 
    * 이 클래스를 사용하여 학습 관련 하이퍼파라미터와 경로 등을 지정할 수 있습니다.
*  Trainer: 
    * 모델을 학습시키고 평가하는 기능을 제공하는 클래스입니다. 
    * Trainer 클래스를 사용하면 간단하게 모델을 학습하고 평가할 수 있습니다. 
    * 학습 및 평가 루프, 데이터로더 및 기타 학습 관련 기능을 관리합니다.

In [ ]:
# AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

# AutoTokenizer, from_pretrained
model_checkpoint = "klue/bert-base"
batch_size = 32
task = "nli"
# tokenizer

In [ ]:
# data_train
# data_val
# data_test

In [ ]:
# data_train.labels 확인


In [ ]:
# sentences


In [ ]:
# BERT 모델의 토크나이저에서 단어 사전 확인
# vocab

### AutoModelForSequenceClassification
AutoModelForSequenceClassification은 입력으로 시퀀스를 받아서 다중 클래스 분류를 수행하는 모델을 생성하는 클래스입니다. from_pretrained() 메서드를 사용하여 이전에 사전 훈련된 모델을 로드합니다. model_checkpoint 매개 변수는 사용할 사전 훈련된 모델을 지정하는 문자열입니다.

num_labels 매개 변수는 모델이 분류해야 하는 라벨의 수를 나타냅니다. 다중 클래스 분류 문제의 경우 라벨의 수는 클래스의 수와 일치해야 합니다.

따라서 위 코드에서는 model_checkpoint에서 불러온 사전 훈련된 모델을 이용하여 입력 시퀀스를 입력받아 7개의 클래스를 분류하는 모델을 생성합니다.

In [ ]:
model_checkpoint

In [ ]:
len(np.unique(data_train.labels))

In [ ]:
# AutoModelForSequenceClassification, from_pretrained
# num_labels 
# model

## BertModel

* https://arxiv.org/abs/1810.04805


<img src="https://i.imgur.com/Eap73Nl.png" width="600">


<img src="https://i.imgur.com/4d0FPsu.png" width="600">

* BertModel (bert)
    * BertEmbeddings: 입력 텍스트의 각 토큰을 고차원 벡터로 변환합니다.
        * word_embeddings: 단어 임베딩, 각 단어를 고정된 크기의 벡터로 매핑합니다.
            * 모델의 어휘 사전에 있는 각 단어를 고정된 크기의 벡터로 변환합니다. 여기서 Embedding(32000, 768)는 어휘 사전에 32,000개의 단어가 있으며, 각 단어는 768차원의 벡터로 표현된다는 것을 의미합니다. padding_idx=0은 특별한 패딩 토큰의 인덱스를 나타내며, 패딩 토큰은 실제 단어를 나타내지 않고 시퀀스의 길이를 조정하는 데 사용됩니다.
        * position_embeddings: 위치 임베딩, 단어의 위치 정보를 벡터로 표현합니다.
            * 위치 임베딩은 모델이 단어의 순서 정보를 포착할 수 있도록 합니다. Embedding(512, 768)은 시퀀스 내 최대 512개의 위치를 각각 768차원 벡터로 표현할 수 있음을 나타냅니다. 이를 통해 모델은 각 단어가 문장 내에서 어디에 위치하는지 인식할 수 있습니다.
        * token_type_embeddings: 문장 유형 임베딩, 주로 두 개의 문장을 입력받는 태스크에서 사용됩니다.
        * LayerNorm and dropout: 정규화와 드롭아웃, 모델의 일반화 능력을 개선합니다.
    * BertEncoder: 입력 데이터에 대한 인코딩을 수행합니다. 여기에는 여러 개의 BertLayer가 포함되어 있습니다.
        * BertLayer:
            * BertAttention: 셀프 어텐션 메커니즘을 통해 입력 데이터의 각 부분이 다른 부분과 얼마나 연관되어 있는지를 학습합니다.
            * BertIntermediate: 피드포워드 신경망, 추가적인 표현 능력을 제공합니다.
            * BertOutput: 인코딩된 정보를 출력하는 레이어입니다.
    * BertPooler: 문장의 전체적인 맥락을 요약하여 고정된 크기의 벡터로 출력합니다.
* Dropout (dropout): 모델의 과적합을 방지하기 위해 사용됩니다.
* Linear (classifier): 인코딩된 텍스트를 분류하기 위한 최종 선형 레이어입니다. 이 예시에서는 출력 차원이 7이므로, 7개의 분류 클래스를 가정하고 있습니다.

In [ ]:
model

### load_metric
Hugging Face Datasets 라이브러리의 load_metric() 함수를 사용하여 GLUE 벤치마크 데이터셋 중 QNLI 데이터셋의 성능 측정 지표(metric)를 로드합니다.

GLUE는 General Language Understanding Evaluation benchmark의 약자로, 자연어 이해(NLU) 과제에 대한 성능을 측정하는 벤치마크입니다. 이 중 QNLI는 Stanford Question Answering Dataset(SQuAD)에서 파생된 데이터셋으로, 두 개의 문장을 입력으로 받아 첫 번째 문장이 두 번째 문장의 Entailment, Not entailment, Neutral 인지를 분류하는 이진 분류 문제입니다.

* Entailment, not entailment, neutral 은 자연어 처리 분야에서 문장 간의 관계를 나타내는 레이블 중 하나입니다.
* "Entailment"는 주어진 문장이 다른 문장을 함의하는 경우를 나타냅니다. 즉, 첫 번째 문장이 참이면 두 번째 문장도 참이어야 합니다. 예를 들어, "고양이가 먹이를 먹었다"라는 문장이 "동물이 먹이를 먹는다"라는 문장을 함의합니다.
* "Not entailment"는 주어진 문장이 다른 문장을 함의하지 않는 경우를 나타냅니다. 예를 들어, "고양이가 물을 마신다"라는 문장은 "고양이가 먹이를 먹었다"라는 문장을 함의하지 않습니다.
* "Neutral"은 두 문장 사이에 어떤 함의적 관계도 없는 경우를 나타냅니다. 예를 들어, "오늘은 맑은 날씨다"와 "고양이는 살찌고 있다"는 문장 사이에는 함의적 관계가 없습니다.


따라서 load_metric() 함수는 해당 데이터셋의 이름과 평가 지표 이름을 인자로 받아 해당 평가 지표 객체를 반환합니다. 이렇게 반환된 metric 객체를 사용하여 모델 성능을 평가할 수 있습니다.


* https://klue-benchmark.com/leaderboard
    * KLUE-STS (Semantic Textual Similarity): 문장 간 의미적 유사성 측정
    * KLUE-NLI (Natural Language Inference): 주어진 두 문장의 관계 분류 (함의, 모순, 중립)
    * KLUE-NER (Named Entity Recognition): 문장 내 명명된 엔터티 식별 (사람, 장소 등)
    * KLUE-RE (Relation Extraction): 문장에서 엔터티 간의 관계 추출
    * KLUE-DP (Dependency Parsing): 문장 내 단어 간 의존 구조 분석
    * KLUE-MRC (Machine Reading Comprehension): 문맥에서 질문에 대한 정확한 답변 추출
    * WOS (Word Ordering in Sentences): 문장 내 단어 순서 맞추기


In [ ]:
# metric.compute
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return load_metric("glue", "qnli").compute(predictions=predictions, references=labels)

### TrainingArgument
 TrainingArgument는 Hugging Face의 Transformers 라이브러리를 사용하여 fine-tuning을 위한 모델 학습을 설정합니다.

* metric_name: 평가 지표의 이름을 지정합니다.
* args: TrainingArguments 클래스의 객체를 생성합니다. 이 클래스는 모델 훈련 시 필요한 인자를 담고 있습니다. 여기서는 학습 경로, 학습률, 배치 크기, 에폭 수, 가중치 감쇠 등을 설정합니다.
* "test-nli": 학습 경로를 설정합니다.
* evaluation_strategy: 평가 전략을 설정합니다. 여기서는 epoch마다 평가를 수행하도록 설정되어 있습니다.
* learning_rate: 학습률을 설정합니다.
* per_device_train_batch_size: 학습에 사용될 배치 크기를 설정합니다.
* per_device_eval_batch_size: 검증에 사용될 배치 크기를 설정합니다.
* num_train_epochs: 학습할 총 에폭 수를 설정합니다.
* weight_decay: 가중치 감쇠를 설정합니다.
    * weight_decay는 오버피팅을 방지하기 위한 L2 정규화(regularization)의 일종입니다. 정규화는 모델이 학습 데이터에 과적합(overfitting)되는 것을 방지하는 데 사용됩니다.
    * L2 정규화는 가중치(weight) 값이 큰 값을 가지는 경우 이를 작게 만들어서 오버피팅을 방지합니다. 이를 위해 각 가중치의 제곱에 대한 합에 대해 패널티를 부여하여 학습 중 가중치 값이 너무 커지지 않도록 제한합니다. 따라서 weight_decay 값이 클수록 패널티가 강해지고, 가중치의 크기가 줄어들게 됩니다.
    * weight_decay 값을 적절히 설정하면 모델의 일반화 성능을 향상시키는 데 도움이 됩니다. 그러나 값이 너무 크게 설정되면 모델의 학습이 매우 느려질 수 있습니다. 일반적으로 weight_decay 값은 0.01 ~ 0.001 정도의 작은 값으로 설정됩니다.
* load_best_model_at_end: 학습이 끝난 후, 가장 좋은 성능을 보인 모델을 불러올지 여부를 설정합니다.
* metric_for_best_model: 가장 좋은 성능을 보인 모델을 선택하기 위해 사용할 평가 지표의 이름을 설정합니다.


In [ ]:
# TrainingArguments
metric_name = "accuracy"

args = _____________(
    "test-nli",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=False,
    metric_for_best_model=metric_name,
)

### AutoModelForSequenceClassification
AutoModelForSequenceClassification.from_pretrained()는 transformers 라이브러리에서 제공하는 모델 클래스 중 하나로, 미리 학습된 모델 체크포인트를 불러와서 새로운 데이터에 대한 분류 작업을 수행할 수 있게 합니다.

여기서 model_checkpoint는 불러올 모델 체크포인트의 이름이나 경로를 의미하며, num_labels는 분류할 라벨의 개수를 의미합니다. 예를 들어, 10개의 카테고리로 분류하는 작업을 수행한다면 num_labels는 10이 됩니다.

이 함수는 AutoConfig.from_pretrained()와 함께 사용되는 경우가 많습니다. AutoConfig 클래스는 모델 아키텍처 및 하이퍼파라미터를 정의하며, AutoModelForSequenceClassification은 이러한 아키텍처와 하이퍼파라미터를 기반으로 미리 학습된 모델 체크포인트를 로드합니다.

이렇게 불러온 모델은 Trainer 클래스와 함께 사용하여 fine-tuning 작업을 수행할 수 있습니다.

In [ ]:
# AutoModelForSequenceClassification.from_pretrained

def model_init():
    return _____________.from_pretrained(model_checkpoint, num_labels=num_labels)

## Trainer
* Trainer는 데이터 로딩, 전처리, 모델 학습 및 평가 등의 작업을 자동으로 처리해주며, 다양한 모델 아키텍처를 학습시킬 수 있습니다.
* Trainer는 기본적으로 PyTorch Lightning의 Trainer와 유사한 기능을 제공합니다. 학습 데이터와 평가 데이터, 최적화 함수, 학습 속도, 학습 횟수 등을 설정하고 모델을 학습시킵니다.
* Trainer는 모델 평가 및 예측 결과 출력, 학습 과정 시각화, 최상의 모델 저장 및 로드 등의 기능을 제공합니다. 이러한 기능들은 학습 및 평가 프로세스를 자동화하고 모델 학습을 더욱 효율적으로 수행할 수 있도록 도와줍니다.

In [ ]:
# Trainer
trainer = _____________(
    model_init=model_init,
    args=args,
    train_dataset=data_train,
    eval_dataset=data_val,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
# train


## 평가

In [ ]:
# evaluate


## 제출
https://dacon.io/competitions/official/235747/mysubmission

In [ ]:
# predict


In [ ]:
# 파일로 저장
submission = pd.read_csv('data/klue/sample_submission.csv')
submission['topic_idx'] = y_predict
submission.to_csv("klue-bert-base.csv",index=False)

In [ ]:
# 소요시간 표시
elapsed = time.time() - start_time
elapsed / 60

* 참고  
    * https://github.com/huggingface/notebooks/blob/main/examples/text_classification.ipynb  
    * https://dacon.io/en/competitions/official/235747/codeshare/3047
